## Cropping images to individual wells for publication

In [51]:
# Importing the necessary libraries to run scripts

import pandas as pd
import time
import scipy.stats as stats
import numpy as np
import pathlib as plb 
import numpy as np
from skimage.io import imread, imsave, imshow
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import closing, square, remove_small_objects
from skimage import exposure
from skimage.exposure import match_histograms
import matplotlib.pyplot as plt

#### 1. The batch_process() function will iterate and load all of the image files in a folder. 
<p> It also calls the following function get_dims() to identify and capture the dimensions of each well for cropping. </p>

In [52]:
def batch_process(image_fpath, rslt_path):
    image_folder = plb.Path(image_fpath)
    results_folder = plb.Path(rslt_path)
    results_df = pd.DataFrame()
    for i in image_folder.glob('[!._]*.tif*'):
        pattern = "^[a-zA-Z]"
        fname = i.stem
        image_data = get_dims(i, rslt_path, fname)

        #image_data.to_csv(path_or_buf= results_folder.joinpath(fname + '.csv'))
        results_df = results_df.append(image_data)

    return results_df

#### 2. get_dims() identifies all of the wells and their locations in a single image and  organizes them into a dataframe.

In [53]:
#Capturing the slot and well IDs from the metadata
def slots_wells(row):
    row['Slot'] = row['WellNo'][0]
    row['Well'] = row['WellNo'][1]
    return row

def get_dims(flpath, rslt_path, fname):
    label_begin = time.time()
    image = imread(flpath)
    image_nvrt = np.invert(image)
    
    # apply threshold
    print('At threshold')
    thresh = threshold_otsu(image_nvrt)
    print('Threshold: ' + str(thresh))
    bw = closing(image_nvrt > thresh, square(3))

    # remove artifacts connected to image border
    cleared = clear_border(bw)
    #cleared = remove_small_objects(clear_border(bw))
    print('Clearing small objects took ', str(int(time.time() - label_begin)), 'seconds.')

    # label image regions
    label_image = label(bw)
    print('Feature finding and labeling took ', str(int(time.time() - label_begin)), 'seconds.')
    #image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

    props = regionprops_table(label_image, properties=('label','centroid', 'bbox', 'area'))
    dff=pd.DataFrame(props)

    df_area = dff.sort_values(by=['area'], ascending=False)
    image_center = (int(label_image.shape[1]/2),int(label_image.shape[0]/2))


    wells = df_area[(df_area.area>= 2000000) & (df_area.area<=2500000)]
    wells=wells.sort_values(by=['bbox-1'])
    print('Number of wells: ' + str(len(wells)))

    wells.reset_index(drop=True, inplace=True)

    # Sort the plates into the left and right
    mask1 = wells['bbox-1'] > image_center[0]
    df_r = wells[mask1]
    df_l = wells[~mask1]

    ## Split the right plates to upper (#1) and lower (#4)
    dffr=df_r.sort_values(by=['bbox-0'])
    dffr.reset_index(drop=True, inplace=True)


    mask2 = df_r['bbox-0'] > image_center[1]
    dff4 = df_r[mask2]
    dff1 = df_r[~mask2]

    ### Sort the wells on each plate
    df1=dff1.sort_values(by=['bbox-0'])
    df4=dff4.sort_values(by=['bbox-0'])
    df4.reset_index(drop=True, inplace=True)
    df1.reset_index(drop=True, inplace=True)

    ## Split the left plates to upper (#2) and lower (#3)
    dffl=df_l.sort_values(by=['bbox-0'])
    dffl.reset_index(drop=True, inplace=True)

    #MinCol2=1.05*list(dffl.items())[3][1][3]
    mask = df_l['bbox-0'] > image_center[1]
    dff3 = df_l[mask]
    dff2 = df_l[~mask]

    ### Sort the wells on each plate
    df2=dff2.sort_values(by=['bbox-0'])
    df3=dff3.sort_values(by=['bbox-0'])
    df3.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    ## Update the label of each well
    new_label_1 = pd.Series(['1A', '1B','1C','1D'], name='label', index=[0,1,2,3])
    df1.update(new_label_1)

    new_label_2 = pd.Series(['2A', '2B','2C','2D'], name='label', index=[0,1,2,3])
    df2.update(new_label_2)

    new_label_3 = pd.Series(['3A', '3B','3C','3D'], name='label', index=[0,1,2,3])
    df3.update(new_label_3)

    new_label_4 = pd.Series(['4A', '4B','4C','4D'], name='label', index=[0,1,2,3])
    df4.update(new_label_4)


    ### Append the dataframes
    df_f=df1.append(df2, ignore_index=True).append(df3, ignore_index=True).append(df4, ignore_index=True)

    df_f = df_f.rename(columns={'label': 'WellNo'})
    df_f.apply(lambda row: slots_wells(row), axis=1)

    ### Calling the cropWell() function to crop the wells id'd in the dataframe
    results_df = cropWell(df_f, image, flpath, rslt_path, fname)
    return results_df
    
        #save_worm_locations(df_f,filtered_worm,path_rslt,label)

In [54]:
def find_comp(ub, fname):
    comp = metadata.loc[
        (metadata['File Name']==row['File Name']) & 
        (metadata['WellNo']==row['WellNo'])]['Compound']
    #print(pid)
    return comp.values[0]

#### 3. Generating a function to crop each well and save it as an individual image

In [69]:
def cropWell(df_f,image, im_path, path_rslt, fname):
    ub_dat = pd.read_csv('D:/_2021_08_screen/practice_run/data/MS1_analysis.csv')
    #Parsing through the dataframe generated in getDims() 
    for index, row in df_f.iterrows():
        
        #Using the mask dimensions to crop down to individual wells
        fin_image = image[ df_f['bbox-0'][index]:df_f['bbox-2'][index], df_f['bbox-1'][index]:df_f['bbox-3'][index]]
        
        #Assigning the correct well label to the image and saving the file
        wellno = df_f['WellNo'][index]
        image_dims = fin_image.shape
        
        compound = ub_dat.loc[(ub_dat['WellNo'] == wellno) & (ub_dat['File Name'] == fname)]['Compound Name']
        print(compound.values[0])
        imsave((path_rslt + compound.values[0] + '/' + fname + '_' + wellno + '.tiff'), fin_image)

    return df_f


#### 4. Calling the function to initiate cropping images
1. To run the program you only need a single function call: **batch_process(input_folder, output_folder)**
2. Batch process requires 2 arguments: 
    1. A file path to the folder containg the images to be cropped and
    2. A file path to the folder where you would like to store the cropped images.
3. batch_process() also returns a dataframe with the bounding box dimensions for each well that can be saved if the user wants to capture that data

In [72]:
#Input folder file path
img_p = 'D:/_2021_08_screen/practice_run/Images_alt/'
#Output folder file path
r_p = 'D:/_2021_08_screen/practice_run/crops/'
#Function call
dims_df = batch_process(img_p, r_p)


At threshold
Threshold: 124
Clearing small objects took  7 seconds.
Feature finding and labeling took  9 seconds.
Number of wells: 16
EMT
1-Oct
EMT
DI
DI
1-Oct
EMT
DMSO
DMSO
IA
EMT
EMT
EMT
IA
1-Oct
EMT
At threshold
Threshold: 112
Clearing small objects took  7 seconds.
Feature finding and labeling took  9 seconds.
Number of wells: 16
IA
DI
DI
EMT
DI
1-Oct
2no
2no
1-Oct
DI
IA
1-Oct
2no
2no
IA
2no
At threshold
Threshold: 122
Clearing small objects took  7 seconds.
Feature finding and labeling took  9 seconds.
Number of wells: 16
DMSO
2no
IA
1-Oct
DI
2no
2no
DMSO
IA
2no
1-Oct
DI
EMT
2no
1-Oct
IA
At threshold
Threshold: 119
Clearing small objects took  7 seconds.
Feature finding and labeling took  9 seconds.
Number of wells: 16
DMSO
EMT
DMSO
1-Oct
IA
EMT
2no
DMSO
1-Oct
DMSO
IA
EMT
EMT
DMSO
DMSO
DI
At threshold
Threshold: 129
Clearing small objects took  7 seconds.
Feature finding and labeling took  9 seconds.
Number of wells: 16
IA
DI
1-Oct
DI
DMSO
2no
1-Oct
IA
DMSO
2no
DI
DI
IA
EMT
2no
DM

#### 5. Capturing and calculating the average width of each well to use for plotting
All values are represented in pixels. Images are captured in 1200 dpi which will be utilized to translate image data into milimeters.

In [9]:
dims_df['Width']=dims_df.apply(
    lambda row: row['bbox-3'] - row['bbox-1'], axis=1)


In [10]:
 dims_df.to_csv('D:/_2021_08_screen/practice_run/crops/well_dims.csv')

In [50]:
image_crops = plb.Path('D:/_2021_08_screen/practice_run/crops/')
reference = imread('D:/_2021_08_screen/practice_run/crops/MS1_004_4A.tiff')
for im in image_crops.glob('[!._]*.tif*'):
    image = imread(im)
    fname = im.stem
    matched = match_histograms(image, reference)
    new_match = matched.astype(np.uint8)
    imsave('D:/_2021_08_screen/practice_run/matched/' + fname + '.tiff', new_match)